# Case Study: Is there a relationship between playground availability and household income in New York City?

First, we import the relevant Python libraries and load the relevant datasets. The parks and playground datasets were taken from the NYC Open Data website. The income dataset was taken from the IRS website. <br>
<br>
Directory of Parks - Updated September 2018 <br>
https://data.cityofnewyork.us/Recreation/Directory-of-Parks/79me-a7rs <br>
<br>
Directory of Playgrounds - Updated January 2019 <br>
https://data.cityofnewyork.us/Environment/Directory-of-Playgrounds/59gn-q4ai <br>
<br>
Individual Income Tax Statistics by Zip Code - Updated 2016 <br>
https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi <br>
<br>
<br>
For optimal results, run the cells in sequence. This notebook requires numpy, pandas, and folium packages.

In [2]:
import numpy as np
import pandas as pd

In [3]:
df_park = pd.read_json('DPR_Parks_001.json')
df_playground = pd.read_json('DPR_Playgrounds_001.json')
df_income = pd.read_csv('16zpallagi.csv')

# Cleaning the Parks and Playgrounds Datasets

There are 1673 total rows in the Parks dataset. However, the Parks dataset is incomplete. 1442 parks are labeled with a zipcode. 125 parks have multiple zip codes. 106 parks have no zipcodes. Each park has a unique identifier, Prop_ID. 

There are 1269 total rows in the Playgrounds dataset.  All 1269 playgrounds have a Prop_ID that would link them to a park. However, 234 school playgrounds have Prop_IDs formatted differently than those of regular parks. 1245 playgrounds have latitude and longitude data. Furthermore, the Playgrounds dataset does not include a zip code column, but does include address data, some of which may include a zip code. Since not all playgrounds have zip codes or have matching Prop_IDs, the best way to identify each playground is by its name. 

In [4]:
#extract the zipcode from the address found in Location column
df_playground['zipcode'] = df_playground['Location'].str.extract('(\d\d\d\d\d)', expand=True)
df_playground[df_playground['School_ID']=='K006']


,Accessible,Adaptive_Swing,Level,Location,Name,Playground_ID,Prop_ID,School_ID,Status,lat,lon,zipcode
1003,Y,N,4.0,"43 Snyder Ave, Brooklyn, NY 11226",PS 6K,None,B,K006,Open to the Public,40.6489,-73.9566,11226


We have to figure out a way to combine the Parks and Playgrounds datasets. We can merge the datasets using an Outer Join, linking each dataset by the Prop_ID column that they both share, while preserving any other data that would have been lost from an inner join. Let's call this the AllParks dataset.

Ultimately, even though not all parks and playgrounds have zipcodes associated with them, the Zipcode column is the best way to link the AllParks dataset with the Income dataset. Because of this, not every park and playground will be included in the final result.

For the purposes of this analysis, the combined park and playground dataset will include 1413 parks and 1037 playgrounds.

In [5]:
#combine the park and playground datasets based on their zip codes. Not every park and playground will be included
#due to the fact that some parks and playgrounds don't have zip codes.

df_allparks = pd.merge(df_park,df_playground,on='Prop_ID',how='outer')

#Combine the zipcodes from the park dataset with the zipcodes from the playground dataset into one column
df_allparks = pd.concat([df_allparks['Name_x'],df_allparks['Name_y'],df_allparks['Zip'].combine_first(df_allparks['zipcode'])],axis=1)

#Because of the outer join, we only want to count unique names for parks and playgrounds to avoid overcounting
df_allparks = df_allparks.groupby('Zip', as_index=False).agg({'Name_x':'nunique','Name_y':'nunique'}).sort_values(by=['Name_x'],ascending=False)

#Rename the columns
df_allparks = df_allparks.rename(columns = {'Zip':'Zipcode','Name_x':'No. of Parks','Name_y':'No. of Playgrounds'})
df_allparks = df_allparks[['Zipcode','No. of Parks','No. of Playgrounds']]

#Because some rows have multiple zipcodes, we could eliminate them because they won't fit with our income dataset
#create a new column that counts the number of characters in the Zipcode column. A character count 
#greater than 5 indicates the row has more than one zipcode
df_allparks['counts'] = df_allparks['Zipcode'].str.len()
df_allparksCompatible = df_allparks[df_allparks['counts']==5]

print('Count of Parks and Playgrounds (multiple zipcodes):')
print('\n')
print(df_allparks.sum())
print('\n')
#there are 1537 parks and 1208 playgrounds in our cleaned-up AllParks dataset
#this dataset includes parks/playgrounds with multiple zipcodes

print('\n')
print('Count of Parks and Playgrounds (multiple zipcodes removed):')
print('\n')
print(df_allparksCompatible.sum())
#there are 1413 parks and 1037 playgrounds in the Compatible AllParks dataset
#this dataset excludes parks/playgrounds with multiple zipcodes and is more compatible
#with comparing against income dataset


Count of Parks and Playgrounds (multiple zipcodes):


Zipcode               1121111201100021121210314113771030111206104571...
No. of Parks                                                       1537
No. of Playgrounds                                                 1208
counts                                                             2916
dtype: object




Count of Parks and Playgrounds (multiple zipcodes removed):


Zipcode                       inf
No. of Parks          1413.000000
No. of Playgrounds    1037.000000
counts                 860.000000
dtype: float64


# Cleaning the Income Dataset
The IRS Income dataset includes data from all 50 US states. We only want income data from New York, so we filter the dataset.


In [6]:
df_incomeNY = df_income[df_income['STATE'] == 'NY']
df_incomeNY

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
102602,36,NY,0,1,3445320,2355000,445210,595120,2114350,4860320,...,1713210,1659729,20,8,0,0,441650,387884,2729330,5662377
102603,36,NY,0,2,2124110,1162700,441680,474330,1285810,3760990,...,1816240,5000576,0,0,0,0,290630,482940,1801120,4662872
102604,36,NY,0,3,1297620,644300,399250,216270,829800,2372130,...,1257120,7572716,30,18,20,42,258810,585359,1013190,2932751
102605,36,NY,0,4,825090,307120,392040,99830,554500,1705160,...,816050,7895883,0,0,0,0,183170,543754,615490,2209076
102606,36,NY,0,5,1242910,293570,841180,82060,877120,3053270,...,1237140,23154728,16500,4382,3910,2224,343770,1668063,840920,4104126
102607,36,NY,0,6,533020,86040,419810,11670,422720,1477580,...,532690,86295410,309570,959747,319850,2085711,246650,6342381,193610,6683628
102608,36,NY,6390,1,40,50,0,0,30,40,...,40,97,0,0,0,0,0,0,40,73
102609,36,NY,6390,2,20,0,30,0,0,40,...,0,0,0,0,0,0,0,0,0,0
102610,36,NY,6390,3,50,30,0,0,30,80,...,50,388,0,0,0,0,40,134,40,115
102611,36,NY,6390,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We only need three columns from this dataset for our analysis:
    - Zipcode
    - Income Bracket (agi_stub column)
    - Number of Returns for each bracket (N1 column)

In [7]:
#Group the NY income data by Zipcode, Income Bracket, and Number of Returns (frequency)
df_incomeNY = df_incomeNY.groupby(['zipcode','agi_stub','A02650'], as_index=False).agg({'N1':'sum'})
df_incomeNY

,zipcode,agi_stub,A02650,N1
0,0,1,42899339,3445320
1,0,2,78336378,2124110
2,0,3,80771436,1297620
3,0,4,72245827,825090
4,0,5,172448013,1242910
5,0,6,348418312,533020
6,6390,1,537,40
7,6390,2,818,20
8,6390,3,3309,50
9,6390,4,0,0


Now, we should check for how complete this NY income dataset is. Bear in mind that the dataset includes data for residents outside of the five boroughs of New York City as well. <br>
<br>
Using the list of zip codes from the NYC Open Data site, we compare the list of NYC zipcodes against the NY income dataset. We find that there are 4,281,820 tax returns from NYC in the dataset, which is approximately half of the actual NYC population of 8.6 million. <br> 
<br>
According to 2017 estimates of age distribution in New York City by Baruch College, approximately 57.2% of the population are ages 20 to 59 years old. That means nearly 4.9 million people are assumed/eligible to have income tax returns. While the income dataset does not quite match the actual figures of people in New York City, it is still reasonable to conclude that the dataset serves as a good statistical representation of income distribution for NYC.
<br>
<br>
Zip Code Breakdowns for NYC - Updated September 2018<br>
https://data.cityofnewyork.us/City-Government/Zip-code-breakdowns/6bic-qvek
<br>
<br>
Age Distribution of NYC - Updated 2017 <br>
https://www.baruch.cuny.edu/nycdata/population-geography/age_distribution.htm

In [7]:
#list of NYC zipcodes
cols2 = ["10001","10002","10003","10004","10005","10006","10007","10009","10010","10011","10012","10013","10014",
         "10016","10017","10018","10019","10020","10021","10022","10023","10024","10025","10026","10027","10028",
         "10029","10030","10031","10032","10033","10034","10035","10036","10037","10038","10039","10040","10044",
         "10045","10065","10115","10119","10128","10154","10278","10280","10301","10302","10303","10304","10305",
         "10306","10307","10309","10310","10312","10314","10451","10452","10453","10454","10455","10456","10457",
         "10458","10459","10460","10461","10462","10463","10464","10465","10466","10467","10468","10469","10471",
         "10472","10473","10474","10475","10514","10543","10553","10573","10701","10705","10911","10965","10977",
         "11001","11021","11050","11101","11102","11103","11104","11105","11106","11111","11112","11201","11202",
         "11203","11204","11205","11206","11207","11208","11209","11210","11211","11212","11213","11214","11215",
         "11216","11217","11218","11219","11220","11221","11222","11223","11224","11225","11226","11228","11229",
         "11230","11231","11232","11233","11234","11235","11236","11237","11238","11239","11252","11354","11355",
         "11356","11357","11358","11360","11361","11362","11364","11365","11366","11367","11368","11369","11370",
         "11371","11372","11373","11374","11375","11377","11378","11379","11385","11411","11412","11413","11414",
         "11415","11416","11417","11418","11419","11420","11421","11422","11423","11424","11425","11426","11427",
         "11428","11429","11430","11431","11432","11433","11434","11435","11436","11439","11451","11471","11510",
         "11548","11566","11577","11580","11598","11629","11691","11692","11693","11694","11695","11731","11798",
         "11968","12423","12428","12435","12458","12466","12473","12528","12701","12733","12734","12737","12750",
         "12751","12754","12758","12759","12763","12764","12768","12779","12783","12786","12788","12789","13731",
         "16091","20459"]

test = df_incomeNY[df_incomeNY['zipcode'].isin(cols2)]
test.sum()

zipcode      14203062
agi_stub         4494
A02650      360250074
N1            4281820
dtype: int64

The 1-6 scale used in the agi_stub column isn't very helpful for the analysis. So we will replace this with the actual income brackets that the IRS dataset utilizes.

Income Brackets:
    - $1 - $24,999
    - $25,000 - $49,999
    - $50,000 - $74,999
    - $75,000 - $99,999
    - $100,000 - $199,999
    - >$200,000

In [8]:
#Add the official income bracket quantities to replace the 1-6 scale
bins=[0,1,2,3,4,5,np.inf]
names=['1-24999','25000-49999','50000-74999','75000-99999','100000-199999','>200000']
df_incomeNY['income_bracket'] = pd.cut(df_incomeNY['agi_stub'], bins, labels=names)
df_incomeNY = df_incomeNY.rename(columns = {'zipcode':'Zipcode','A02650': 'Total Income'})
df_incomeNY['Act. Mean Income'] = df_incomeNY['Total Income']/df_incomeNY['N1']
df_incomeNY['Act. Mean Income'] = df_incomeNY['Act. Mean Income'].apply(lambda x:x*1000).round(2)

df_incomeNY

,Zipcode,agi_stub,Total Income,N1,income_bracket,Act. Mean Income
0,0,1,42899339,3445320,1-24999,12451.48
1,0,2,78336378,2124110,25000-49999,36879.62
2,0,3,80771436,1297620,50000-74999,62245.83
3,0,4,72245827,825090,75000-99999,87561.15
4,0,5,172448013,1242910,100000-199999,138745.37
5,0,6,348418312,533020,>200000,653668.37
6,6390,1,537,40,1-24999,13425.00
7,6390,2,818,20,25000-49999,40900.00
8,6390,3,3309,50,50000-74999,66180.00
9,6390,4,0,0,75000-99999,NaN


In [9]:
df_incomeNY=df_incomeNY.groupby('Zipcode')['Total Income','N1'].agg('sum')
df_incomeNY.reset_index()
df_incomeNY['average'] = df_incomeNY['Total Income']/df_incomeNY['N1']
df_incomeNY['average'] = df_incomeNY['average'].apply(lambda x:x*1000).round(2)
df_incomeNY = df_incomeNY.reset_index()
df_incomeNY.Zipcode = df_incomeNY.Zipcode.astype(str)
df_incomeNY.columns=['Zipcode','Total Income','Number of Returns','Average Income']
df_incomeNY

,Zipcode,Total Income,Number of Returns,Average Income
0,0,795119305,9468070,83979.03
1,6390,11750,150,78333.33
2,10001,2323084,14520,159992.01
3,10002,2313723,42180,54853.56
4,10003,6720746,28660,234499.16
5,10004,830828,2480,335011.29
6,10005,3171561,5940,533932.83
7,10006,467213,2370,197136.29
8,10007,3048670,3440,886241.28
9,10009,2611193,32410,80567.51


Alternative:

We want to find the average income for each zipcode in order to measure wealth in an area. However, since we only have data on the _number_ of returns for each income bracket, it is impossible to find the actual average income in each zipcode. Thus, the best we can do is to estimate average income by taking the average, or midpoint, value of each income bracket range.

For example, if the income breakdown in a zipcode is as follows:
    - 50 returns in the $25,000-$49,999 income bracket
    - 30 returns in the $50,000-$74,999 income bracket
    - 60 returns in the $75,000-$99,999 income bracket
    - 20 returns in the >$200,000 income bracket
The average income for the zipcode would be calculated as:  

\begin{equation*}
\frac{(\frac{25000+49999}{2}*50) + (\frac{50000+74999}{2}*30) + (\frac{75000+99999}{2}*60) + (\frac{200000+300000}{2}*20)}{(50+30+60+20)} = 87499.56
\end{equation*}

Because the '>\\$200,000' income bracket has no upper limit, we capped the bracket at '\$300,000' so as to not skew average incomes too much to the higher end of the spectrum.
To calculate the median income, we also utilize the midpoint of each income bracket range. Thus, calculated median figures will be inaccurate since there are only 6 possible values, derived from the midpoints of 6 immutable income brackets. But it helps to get an idea of approximately the range in which the median income falls.

We create our own functions to calculate mean and median income.

In [ ]:
#create custom functions to calculate mean and median incomes based on the income brackets.
#the result will be an estimate since we use the midpoint of each income bracket.

lower_limit = [1,25000,50000,75000,100000,200000]
upper_limit = [24999,49999,74999,99999,199999,300000]

def intervalmean(low, high, freq, n):
    mid=[0.0]*n
    sum=0
    freqSum=0
    
    #calculate sum of frequency, sum of multiplication of interval mid-values and respective interval frequency
    for i in range(0,n):
        mid[i] = ((low[i]+high[i])/2)
        sum=sum+mid[i]*freq[i]
        freqSum = freqSum + freq[i]
    return sum/freqSum

def intervalmedian(low, high, freq, n):
    mid=[0.0]*n
    mp = []
    
    #build the array to determine median
    for i in range(0,n):
        mid[i] = ((low[i]+high[i])/2)
        mp.append(mid[i])
    
    newlist = (list(zip(mp,freq)))
    a,b = np.array(newlist, dtype=np.int64).T
    data = np.repeat(a,b)
    return np.median(data)
    



Now, we apply the mean and median functions to our NY income dataset

In [ ]:
#apply the custom mean and median functions to the NY income dataset

df_incomeNY2 = df_incomeNY.groupby(['Zipcode'])['N1'].apply(list)

df_incomeNY2=pd.DataFrame(df_incomeNY2).reset_index()

df_incomeNY2['Mean Income'] = df_incomeNY2.apply(lambda x: intervalmean(lower_limit,\
                                                                        upper_limit,\
                                                                        x['N1'],\
                                                                        len(x['N1'])),axis=1).astype(float).round(2)

df_incomeNY2['Median Income'] = df_incomeNY2.apply(lambda x: intervalmedian(lower_limit,\
                                                                            upper_limit,\
                                                                            x['N1'],\
                                                                            len(x['N1'])),axis=1).astype(float).round(2)

df_incomeNY2.columns=['Zipcode','Bracket Frequencies','Est. Mean Income','Est. Median Income']
df_incomeNY2.Zipcode = df_incomeNY2.Zipcode.astype(str)
df_incomeNY2

The average income for NY State is about \\$60,764. This figure is comparable to the actual US Census figure for median income in NY State in 2017: $62,765 <br>
<br>
US Census Data for New York - Updated 2017 <br>
https://www.census.gov/quickfacts/NY

In [ ]:
#summary statistics for the NY income dataset. This includes data outside of the 5 boroughs of NYC
df_incomeNY.describe()

In [ ]:
df_incomeNY2.describe()

# Finding Correlation Between Income and Playground Availability
Now that we cleaned up the park and playground datasets, as well as the NY income dataset, we can use the utilize the data to determine if there is a relationship between income and playground availibility in a zip code.

After we merge the NY income dataset with the Park/Playground compatible dataset, we get a DataFrame that shows the Mean and Median Incomes, as well as the number of Parks and Playgrounds, for each zipcode.

In [10]:
df_aggregate = pd.merge(df_incomeNY,df_allparksCompatible,on='Zipcode',how='outer')
df_aggregate = df_aggregate.sort_values(by=['No. of Parks'],ascending=False)
df_aggregate

,Zipcode,Total Income,Number of Returns,Average Income,No. of Parks,No. of Playgrounds,counts
254,11211,2360597.0,30950.0,76271.31,40.0,13.0,5.0
245,11201,6307963.0,31970.0,197308.82,27.0,10.0,5.0
3,10002,2313723.0,42180.0,54853.56,25.0,23.0,5.0
255,11212,1196980.0,38370.0,31195.73,22.0,19.0,5.0
61,10314,2898644.0,40740.0,71149.83,21.0,12.0,5.0
50,10301,1251042.0,18080.0,69194.80,20.0,8.0,5.0
302,11377,1944107.0,45490.0,42737.02,20.0,7.0,5.0
249,11206,1483562.0,36490.0,40656.67,20.0,18.0,5.0
68,10457,876847.0,32720.0,26798.50,19.0,15.0,5.0
67,10456,1065238.0,40390.0,26373.81,18.0,12.0,5.0


We generate a statistical correlation matrix to determine the correlation between each column: Mean Income, Median Income, Number of Parks, and Number of Playgrounds. The matrix generates Pearson correlation coefficients, which indicate the linear correlation between two variables.

Pearson correlation coefficients range from -1 to 1. <br>
-1 means that there is a perfectly negative correlation. <br>
+1 means that there is a perfectly positive correlation. <br>
0 means that there is no correlation.

In [11]:
corrmatrix = df_aggregate.corr()
corrmatrix=corrmatrix.drop(['counts'],axis=1)
corrmatrix=corrmatrix.drop(['counts'],axis=0)
corrmatrix

,Total Income,Number of Returns,Average Income,No. of Parks,No. of Playgrounds
Total Income,1.000000,0.999173,0.029954,0.003883,-0.129877
Number of Returns,0.999173,1.000000,0.005205,0.440435,0.646595
Average Income,0.029954,0.005205,1.000000,-0.218538,-0.398991
No. of Parks,0.003883,0.440435,-0.218538,1.000000,0.726582
No. of Playgrounds,-0.129877,0.646595,-0.398991,0.726582,1.000000


As we can see from the results, there is a __moderately strong negative correlation__ between Mean Income and the Number of Playgrounds in any given area in NYC. <br>
>Correlation between Mean Income and Number of Playgrounds: -0.399 <br>
> Correlation between Total Income and Number of Playgrounds: -0.13 <br>
<br  />
> Correlation between Mean Income and Number of Parks: -0.219 <br>
> Correlation between Total Income and Number of Parks: 0 <br>

This means that as average income increases in a zipcode, the number of playgrounds decreases.
While there is evidence of a relationship between income and playground availability, the relationship between income and park availability is still inconclusive due to their weakly negative correlation coefficients.

# Data Visualization - Choropleth Map
To visualize the datasets used in this analysis, we can create a choropleth map. Through color coding and shading, a choropleth map is a great way to see the differences in income or number of parks/playgrounds in any given area. <br>
<br>
First we import the relevant libraries.

In [13]:
import folium
#import json
#from IPython.display import HTML, display

The following code will generate an interactive choropleth map of New York City.<br>
An HTML file labeled 'NYC Choropleth Map' will generate in the folder. This is where the map will be published. The file can be opened in a browser.<br>
<br>
Using the New York Zip Code GeoJson file from Github user enactdev, the zip code area outlines were shaped in the map. The shape file was originally sourced from the US Census Bureau site. <br> 
https://github.com/OpenDataDE/State-zip-code-GeoJSON <br>
<br>
https://www.census.gov/cgi-bin/geo/shapefiles2010/main <br>
<br>
The interactive map will have three overlays: Park Density Map, Income Map, and Playground Map.
Feel free to toggle the overlay views to see a certain map. For the Income Map, you can hover over any area to see more information regarding mean and median incomes in the area. For the Park Density Map, you can hover over any area to see more information regarding the number of parks and playgrounds in that area. The individual red dots from the Playground Map indicate playground locations.

__Important information:__
In the original playground dataset, 1245 playgrounds had data on longitude and latitude, even though most did not have zipcode data. The Playground Choropleth Map was generated using this longitude and latitude information. However, the Park Density Map was generated using the merged dataset between the NY Income and the Park/Playground compatible dataset. This dataset had a total of 1037 playgrounds and are linked to zipcodes. Thus, there are discrepancies in the playground figures between the Park Density Map and the Playground Map.

In [21]:
import json
data = json.load(open('parkagg.json'))
#data
df = pd.DataFrame(data)
df

,Zip
10001,5
"10001, 10011, 10014",1
"10001, 10018",1
10002,25
10003,10
10004,3
"10004, 10280",1
10005,2
10007,3
"10007, 10013, 10280",1


In [30]:
import geopandas as gpd
zips = gpd.read_file('ny_new_york_zip_codes_geo.min.json')

In [32]:
zips2 = pd.merge(zips, df_aggregate, left_on = 'ZCTA5CE10', right_on='Zipcode',how='left')
zips2 = zips2.drop(['Zipcode'],axis=1)
zips2 = zips2.rename(columns={'ZCTA5CE10':'ZipCode'})
zips2

,STATEFP10,ZipCode,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry,Total Income,Number of Returns,Average Income,No. of Parks,No. of Playgrounds,counts
0,36,12205,3612205,B5,G6350,S,40906445,243508,+42.7187855,-073.8292399,N,"POLYGON ((-73.87051700000001 42.751227, -73.86...",850274.0,14810.0,57412.15,NaN,NaN,NaN
1,36,12009,3612009,B5,G6350,S,135241924,2168637,+42.6975663,-074.0355422,N,"POLYGON ((-74.10891100000001 42.653004, -74.10...",336390.0,3800.0,88523.68,NaN,NaN,NaN
2,36,14804,3614804,B5,G6350,S,144718714,232123,+42.3172588,-077.8479358,N,"POLYGON ((-77.92747 42.347754, -77.92631799999...",33185.0,610.0,54401.64,NaN,NaN,NaN
3,36,14836,3614836,B5,G6350,S,77612958,131305,+42.5429182,-077.8781933,N,"(POLYGON ((-77.95599300000001 42.474325, -77.9...",23958.0,470.0,50974.47,NaN,NaN,NaN
4,36,14536,3614536,B5,G6350,S,47193482,425175,+42.5439751,-078.0836709,N,"POLYGON ((-78.050297 42.538502, -78.0502429999...",12597.0,280.0,44989.29,NaN,NaN,NaN
5,36,10464,3610464,B5,G6350,S,9070627,236605,+40.8677868,-073.7999204,N,"(POLYGON ((-73.78619399999999 40.873886, -73.7...",186160.0,2170.0,85788.02,3.0,1.0,5.0
6,36,10470,3610470,B5,G6350,S,3689950,3918,+40.8895273,-073.8726596,N,"POLYGON ((-73.865635 40.902086, -73.865268 40....",389104.0,7660.0,50796.87,2.0,1.0,5.0
7,36,10455,3610455,B5,G6350,S,1844518,0,+40.8147100,-073.9085917,N,"POLYGON ((-73.917119 40.817512, -73.9182140000...",500179.0,17780.0,28131.55,10.0,9.0,5.0
8,36,10473,3610473,B5,G6350,S,5645604,13868,+40.8186904,-073.8584741,N,"POLYGON ((-73.884328 40.822614, -73.8844420000...",1080659.0,27910.0,38719.42,10.0,7.0,5.0
9,36,13797,3613797,B5,G6350,S,86238872,282995,+42.3336646,-076.0404014,N,"POLYGON ((-75.98262 42.345787, -75.98309399999...",40710.0,900.0,45233.33,NaN,NaN,NaN


In [33]:
zips2.to_file('ny_new_york_zip_codes4_geo.min.json', driver='GeoJSON')

In [35]:
#initialize the geojson file
district_geo = r'ny_new_york_zip_codes4_geo.min.json'

#setup the longitude and latitude data from the original playground dataset
df_playground_available = df_playground[df_playground.lat.notnull() & df_playground.lon.notnull()]

#setup the park count data from the original park dataset
data_parks = pd.DataFrame(df_park['Zip'].value_counts())
data_parks.to_json('parkagg.json')
data_parks=data_parks.reset_index()
data_parks.columns=['ZipCode','No_Of_Parks']

#setup the mean income data from the aggregate dataset
data_income = df_aggregate.groupby('Zipcode',as_index=False).agg({'Average Income':'sum'})
data_income.columns=['ZipCode','Mean_Income']
data_income.to_json('incomeagg.json')

#set up the NYC area
map1 = folium.Map(location = [40.75, -73.99], zoom_start=12)

#plot choropleth map of park density
chloropleth = folium.Choropleth(
    geo_data = district_geo,
    data_out = 'parkagg.json',
    data=data_parks,
    columns = ['ZipCode','No_Of_Parks'],
    key_on='feature.properties.ZipCode',
    fill_color = 'YlGn',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Number of parks per Zip',
    name = 'Park Density Map',
    overlay = True,
    highlight = True).add_to(map1)

#plot choropleth map of income density
chloropleth2 = folium.Choropleth(
    geo_data = district_geo,
    data_out = 'incomeagg.json',
    data = data_income,
    columns = ['ZipCode','Mean_Income'],
    key_on = 'feature.properties.ZipCode',
    fill_color = 'PuBu',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Mean Income per Zip',
    name = 'Income Map',
    overlay = True,
    highlight = True).add_to(map1)

#plot available playgrounds based on their coordinates
playgrounds = folium.FeatureGroup(name='Playground Map').add_to(map1)
for i,row in df_playground_available.iterrows():
    folium.CircleMarker((row.lat,row.lon), radius=3, weight=2, 
                        color='red', fill_color='red', fill_opacity=.5, 
                        popup=row.Name, tooltip=row.Name).add_to(playgrounds)

folium.LayerControl(collapsed=False).add_to(map1)

#add hover effects for both maps
chloropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['ZipCode','No. of Parks','No. of Playgrounds']))
chloropleth2.geojson.add_child(
    folium.features.GeoJsonTooltip(['ZipCode','Average Income','Total Income']))


#display(map1)
map1.save('NYC Choropleth Map 2.html')

Done!
<br>
<br>
<br>
# Some Double-Checks

The following code is to test and compare our initial NY Income dataset (that includes income data for all of NY state) with a NYC Income dataset that explicitly filters for NYC zipcodes, in order to check for any discrepancies. Ultimately, the correlation results bear no significant discrepancies from the initial correlation results.

Repeat the setup process like before using a test dataset:

In [ ]:
bins=[0,1,2,3,4,5,np.inf]
names=['1-24999','25000-49999','50000-74999','75000-99999','100000-199999','>200000']
test['income_bracket'] = pd.cut(test['agi_stub'], bins, labels=names)
test = test.rename(columns = {'zipcode':'Zipcode'})
test = test.groupby(['Zipcode'])['N1'].apply(list)

test=pd.DataFrame(test).reset_index()

test['Mean Income'] = test.apply(lambda x: intervalmean(lower_limit,upper_limit,x['N1'],len(x['N1'])),axis=1)
test['Median Income'] = test.apply(lambda x: intervalmedian(lower_limit,upper_limit,x['N1'],len(x['N1'])),axis=1)
test.columns=['Zipcode','Bracket Frequencies','Est. Mean Income','Est. Median Income']
test.Zipcode = test.Zipcode.astype(str)
test

Find the zip code with the highest mean income

In [ ]:
test.sort_values(by='Est. Mean Income', ascending=False)

We have a new NYC-specific merged dataset. The DataFrame is sorted in descending order for Number of Parks

In [ ]:
test2 = pd.merge(test,df_allparksCompatible,on='Zipcode',how='outer')
test2 = test2.sort_values(by=['No. of Parks'],ascending=False)
test2

## Summary Statistics for NYC Dataset
We find that the average income in NYC is \\$62,886, which is comparable with the actual US Census figure for median income in NYC in 2017: $57,782.

In [ ]:
print(test2.describe())
print('\r')
print(test2.info())
print('\r')
print(test2.sum())

Generate a correlation matrix to see the correlation relationships between Estimated Mean Income, Estimated Median Income, Number of Parks, and Number of Playgrounds. The results are similar to those from the first time! This is likely due to the fact that correlation cannot be computed when comparing NaN values. So even though the original NY income dataset had data regarding zip codes outside of NYC, they were not used to calculate correlation since they did not have park and playground data (exclusive to NYC) to compare against.<br>
<br>
There is a weakly negative relationship between income and park availability and a moderately strong negative relationship between income and playground availability.

In [ ]:
corrmatrix2 = test2.corr()
corrmatrix2=corrmatrix2.drop(['counts'],axis=1)
corrmatrix2=corrmatrix2.drop(['counts'],axis=0)
corrmatrix2

Some other interesting findings. This shows the number of parks and playgrounds with more than one zip code. They were excluded from analysis due to overcounting concerns.

In [ ]:
df_allparksCompatible2 = df_allparks[df_allparks['counts']>5]
df_allparksCompatible2.sum()

In [ ]:
df_allparksCompatible2